In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import numpy as np

In [2]:
app = dash.Dash()

In [3]:
import os
server = app.server
server.secret_key = os.environ.get('SECRET_KEY', 'my-secret-key')

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/csv/datasets/infert.csv')

In [5]:
colors = {
    'background': '#E9E9E9',
    'background_a': '#FAFAFA',
    'text': '#000066'
}

In [6]:
markdown_text ='''\
\
\
# Infertility after Spontaneous and Induced Abortion


### Data visualization


'''

In [7]:
markdown_text_a = '''
\
This application serves to visualize [data](https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/csv/datasets/infert.csv) from matched case-control study _Infertility after Spontaneous and Induced Abortion_.
In this study, number of induced and spontaneous abortions, parity, age and education data was collected. You can visualize those data by choosing from three different types of plots.'''

In [8]:
all_options = {
    'Barplot': ['induced abortion', 'spontaneous abortion', 'education'],
    'Scatterplot': [],
    'Histogram': ['age', 'parity']
}

In [9]:
scatter_options = {
    'Barplot': [],
    'Scatterplot': ["education","age","parity","induced","case","spontaneous","stratum","pooled.stratum"],
    'Histogram': []
}

In [10]:
app.layout=html.Div(children=[
    html.Br(),
    html.Div(dcc.Markdown(children=markdown_text), style={
            'textAlign': 'center',
            'color': colors['text'],
    'backgroundColor': colors['background']
        }),
    html.Div(dcc.Markdown(children=markdown_text_a), style={
        'textAlign': 'center',
        'color': colors['text'],
    'backgroundColor': colors['background_a'],
    'marginLeft': 180,
    'marginRight': 180}
            ),
    html.Br(),
    html.Label('Plot options:'),
    dcc.Dropdown(
        id='Graph_dropdown',
        options=[{'label': k, 'value': k} for k in all_options.keys()],
        value='Barplot'
    ),

    dcc.RadioItems(id='variables'),
    html.Hr(),
    
    dcc.RadioItems(id='xaxis-data',style={ 'float': 'left','width': '20%', 'display': 'inline-block'}),
    
    dcc.RadioItems(id='yaxis-data', style={'float': 'middle','width': '45%','display': 'inline-block'}),
    
    html.Hr(),
    
    dcc.Graph(id='Graph'),
   ]
)


In [11]:
@app.callback(
    dash.dependencies.Output('variables', 'options'),
    [dash.dependencies.Input('Graph_dropdown', 'value')])
def set_graph_options(selected_graph):
    return [{'label': i, 'value': i} for i in all_options[selected_graph]]
@app.callback(
    dash.dependencies.Output('variables', 'value'),
    [dash.dependencies.Input('variables', 'options')])
def set_graph_value(available_options):
    return available_options[0]['value']


In [12]:
@app.callback(
    dash.dependencies.Output('xaxis-data', 'options'),
    [dash.dependencies.Input('Graph_dropdown', 'value')])
def set_graph_options(selected_x):
    return [{'label': i, 'value': i} for i in scatter_options[selected_x]]
@app.callback(
    dash.dependencies.Output('xaxis-data', 'value'),
    [dash.dependencies.Input('xaxis-data', 'options')])
def set_graph_value(available_options):
    return available_options[0]['value']

In [13]:
@app.callback(
    dash.dependencies.Output('yaxis-data', 'options'),
    [dash.dependencies.Input('Graph_dropdown', 'value')])
def set_graph_options(selected_y):
    return [{'label': i, 'value': i} for i in scatter_options[selected_y]]
@app.callback(
    dash.dependencies.Output('yaxis-data', 'value'),
    [dash.dependencies.Input('yaxis-data', 'options')])
def set_graph_value(available_options):
    return available_options[0]['value']

In [14]:
@app.callback(
    dash.dependencies.Output(component_id='Graph', component_property='figure'),
    [dash.dependencies.Input(component_id='Graph_dropdown', component_property='value'),
     dash.dependencies.Input(component_id='variables', component_property='value'),
     dash.dependencies.Input(component_id='xaxis-data', component_property='value'),
     dash.dependencies.Input(component_id='yaxis-data', component_property='value')])
def update_figure(plot_type, plot_data, xaxis, yaxis):
    
    
    if plot_type == 'Barplot' and plot_data =='induced abortion':
        trace1 = go.Bar(x = df.groupby(['induced', 'case']).size().unstack(level = 1).index, y = df.groupby(['induced', 'case']).size().unstack(level = 1)[0]/165, name = 'control')

        trace2 = go.Bar(x = df.groupby(['induced', 'case']).size().unstack(level = 1).index,y = df.groupby(['induced', 'case']).size().unstack(level = 1)[1]/83, name = 'case')
    
        data = [trace1, trace2]
        layout = go.Layout(title = 'Induced abortion in case/control group',xaxis=dict(title="number of induced abortion (2= two and more)"))
    
    if plot_type == 'Barplot' and plot_data =='spontaneous abortion':
        trace1 = go.Bar(x = df.groupby(['spontaneous', 'case']).size().unstack(level = 1).index, y = df.groupby(['spontaneous', 'case']).size().unstack(level = 1)[0]/165, name = 'control')

        trace2 = go.Bar(x = df.groupby(['spontaneous', 'case']).size().unstack(level = 1).index,y = df.groupby(['spontaneous', 'case']).size().unstack(level = 1)[1]/83, name = 'case')
    
        data = [trace1, trace2]
        layout = go.Layout(title = 'Spontaneous abortion in case/control group',xaxis=dict(title="number of spontaneous abortion (2= two and more)"))
    
    if plot_type == 'Barplot' and plot_data =='education':
        trace1 = go.Bar(x = df.groupby(['education', 'case']).size().unstack(level = 1).index, y = df.groupby(['education', 'case']).size().unstack(level = 1)[0]/165, name = 'control')

        trace2 = go.Bar(x = df.groupby(['education', 'case']).size().unstack(level = 1).index,y = df.groupby(['education', 'case']).size().unstack(level = 1)[1]/83, name = 'case')
    
        data = [trace1, trace2]
        layout = go.Layout(title = 'Education of case/control group',xaxis=dict(title="years in school"))
    
    
    if plot_type == 'Histogram' and plot_data == 'age':
        case = df[df.case == 1].age
        control = df[df.case == 0].age
        
        trace1 = go.Histogram(x=case,opacity=0.5,name = 'case')
        trace2 = go.Histogram(x=control,opacity=0.5,name = 'control')
        
        data = [trace1, trace2]
        layout = go.Layout(barmode = 'stack',title = "age by case/control group",xaxis=dict(title="age"))
        
    if plot_type == 'Histogram' and plot_data == 'parity':
        case = df[df.case == 1].parity
        control = df[df.case == 0].parity
        
        trace1 = go.Histogram(x=case,opacity=0.5,name = 'case')
        trace2 = go.Histogram(x=control,opacity=0.5,name = 'control')
        
        data = [trace1, trace2]
        layout = go.Layout(barmode = 'stack',title = "parity by case/control group",xaxis=dict(title="parity"))
     
    if plot_type == 'Scatterplot':
        traces = []
        for i in df.case.unique():
            df_by_case = df[df['case'] == i]
            traces.append(go.Scatter(
            x=df_by_case[xaxis],
            y=df_by_case[yaxis],
            mode='markers',
            opacity=0.3,
            marker=dict(
                size='16',
                opacity=0.7,
                color = np.random.randn(248),
                colorscale ='Viridis',
                showscale=True)))
        data = traces
        layout = go.Layout()
    
        
    figure={'data':data,'layout':layout} 
    return figure

In [15]:
app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})

In [16]:
if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
